![Logo kursu Python Level Up](https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2020/master/logo.png)

![Plan kursu](https://raw.githubusercontent.com/daftcode/daftacademy-python_levelup-spring2020/master/program.png)

# A jak Asyncio


### Marcin Jaroszewski
### Python level up 2020
### 11.05.2020

# 1. Wstemp teokratyczny

Bardzo polecam książkę: https://www.amazon.com/Modern-Operating-Systems-Andrew-Tanenbaum/dp/013359162X

## 1.1 Ein procesor, ein rdzeń

Jak to było kiedy komputery miały jeden procesor, jakie były ograniczenia, problemy i wyzwania (np `time slicing`).

## 1.2 Ein procesor, viel rdzeni 

Miało być tak pięknie - ale rzeczywistość nas dogoniła.

## 1.3 Proces

W zasadzie: wykonywany program (jeden), można w wielu procesach uruchomić wiele "kopii" tego samego programu. Kontrolowany prez syste operacyjny. 

- pamięć
- czas procesora
- IPC

https://pl.wikipedia.org/wiki/Proces_(informatyka)

## 1.4 Wontek

Są przynajmniej 2 rodzaje:
- systemu operacyjnego
- zielone

Różnice względem procesu:
- pamięć
- czas procesora
- IPC

https://pl.wikipedia.org/wiki/W%C4%85tek_(informatyka)

## 1.5 Concurency

`Concurrency means multiple computations are happening at the same time. Concurrency is everywhere in modern programming` - za https://web.mit.edu/6.005/www/fa14/classes/17-concurrency/

Zauważmy, że w "definicji" nie jest limitowane, gdzie się dzieją "obliczenia".

## 1.6 Paralellism

Współbieżność

```
Concurrency is when two or more tasks can start, run, and complete in overlapping time periods. It doesn't necessarily mean they'll ever both be running at the same instant. For example, multitasking on a single-core machine.

Parallelism is when tasks literally run at the same time, e.g., on a multicore processor.
```
za https://stackoverflow.com/a/1050257

## 1.8 Synchronizacja

- sekcje krytyczne
- locki
- mutexy
- semafory
- inne

## 1.9 Rodzaje obciążeń

- CPU
- IO

# 2. A co na to Python?

## 2.1 `multiprocessing`

https://docs.python.org/3.8/library/multiprocessing.html

## 2.2 `subprocess`

https://docs.python.org/3/library/subprocess.html

## 2.3 `threading`

https://docs.python.org/3/library/threading.html

## 2.4 GIL - Global Interpreter Lock

https://wiki.python.org/moin/GlobalInterpreterLock

jak się ma GIL do:
- threading w Python
- C pod spodem niektórych modułów
- zewnętrznych bibliotek np `numpy`

## 2.5 `asyncio`

https://docs.python.org/3/library/asyncio.html

`Cooperative multitasking` - dzisiejsza nazwa wytrych. 

Przykład z szachami.

# 3. Słowniczek

- event loop
- awaitable
- coroutine
- `async`
- `await`

# 4. Przykłady

Będziemy odpytywać urle z dwóch plików

urls
```
http://httpbin.org/delay/9
http://httpbin.org/delay/8
http://httpbin.org/delay/7
http://httpbin.org/delay/6
http://httpbin.org/delay/5
http://httpbin.org/delay/4
http://httpbin.org/delay/3
http://httpbin.org/delay/2
http://httpbin.org/delay/1
```

urls_2
```
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
http://httpbin.org/delay/1
```

## 4.1 Synchroniczne odpytywanie z requests

```python
import time

import requests


responses = []

urls_f_name = 'urls'
# urls_f_name = 'urls_2'
start = time.time()
with open(urls_f_name, 'r') as f:
    for line in f.readlines():
        print(f'Checking {line}')
        resp = requests.get(line)
        responses.append(resp)
end = time.time()

print(f'elapsed: {end - start} seconds')
```

## 4.2 Asynchrnoniczne odptywanie z requests

Nie da się - bo `requests` jest "blokujące".

O co chodzi z blokowaniem w kontekście biblitek/modułów **WAŻNE**

## 4.3 Asynchroniczne odpytywanie z `httpx` lub `aiohttp` **CODING GONE LIVE** 

- httpx: https://github.com/encode/httpx
- aiohttp: https://docs.aiohttp.org/en/stable/

v. 1
```python
import asyncio
import time

import httpx


responses = []
urls_f_name = 'urls'


async def main():
    with open(urls_f_name, 'r') as f:
        async with httpx.AsyncClient(timeout=15) as client:
            for line in f.readlines():
                print(f'Checking {line}')
                resp = await client.get(line)
                responses.append(resp)    


if __name__ == '__main__':
    start = time.time()
    asyncio.run(main())
    end = time.time()
    print(f'elapsed: {end - start} seconds')

```

v. 2
```python
import asyncio
import time

import httpx


responses = []
urls_f_name = 'urls'
urls_f_name = 'urls_2'

partials = []
async def get(url, client):
    print(f'Checking {url}')
    start = time.time()
    resp = await client.get(url)
    end = time.time()
    partials.append(end-start)
    responses.append(resp)

async def main():
    with open(urls_f_name, 'r') as f:
        async with httpx.AsyncClient(timeout=15) as client:
            coroutines = []
            for line in f.readlines():
                coroutines.append(get(line, client))
            await asyncio.gather(*coroutines)


if __name__ == '__main__':
    start = time.time()
    asyncio.run(main())
    end = time.time()
    print(f'elapsed: {end - start} seconds')
    print(f'total partials: {sum(partials)}')
```

## 4.4 Wyjaśnienie różnic w przykładach i wynikach

# 5. Jak to się ma do apek webowych?

https://12factor.net/

## 5.1 Synchoroniczna apka z jednym workerem 

## 5.2 Asynchroniczna apka z jednym workerem

# 6. Uwagi końcowe

## 6.1 czemu `asleep(0)` bywa dobrym pomysłem, ale nie zawsze

https://stackoverflow.com/questions/55857581/converting-small-functions-to-coroutines/55866425#55866425

https://stackoverflow.com/questions/41932359/timeout-for-python-coroutines/48816319#48816319

```
It means "request value from the provided awaitable object, yielding control to the event loop if the object indicates that it does not have a value ready." The if is crucial: if the object does have a value ready, this value will be used immediately without ever deferring to the event loop. In other words, await doesn't guarantee that the event loop will get a chance to run.
```

# 7. Pytania?

# 8. Wincyjj materiałuff

Real Python:
- https://realpython.com/async-io-python/

Łukasz Langa (bardzo polecam):
1. https://www.youtube.com/watch?v=Xbl7XjFYsN4
2. https://www.youtube.com/watch?v=E7Yn5biBZ58
3. https://www.youtube.com/watch?v=-CzqsgaXUM8
4. ma być jeszcze kilka


# 8. That's all folks